In [1]:
!pip3 install d2l
!pip3 install pillow
!pip3 install pytesseract
!pip3 install transformers
!pip3 install timm

Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------ 112.6/112.6 kB 155.9 kB/s eta 0:00:00
     ---------------------------------------- 9.3/9.3 MB 363.2 kB/s eta 0:00:00
     ---------------------------------------- 7.2/7.2 MB 264.7 kB/s eta 0:00:00
     -------------------------------------- 61.2/61.2 kB 326.8 kB/s eta 0:00:00
     -------------------------------------- 58.8/58.8 kB 258.4 kB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
conda-repo-cli 1.0.20 requires requests==2.28.1, but you have requests 2.25.1 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 2.2/2.2 MB 243.3 kB/s eta 0:00:00
     ------------------------------------ 263.9/263.9 kB 405.8 kB/s eta 0:00:00


In [5]:
!pip3 install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 86.0/86.0 kB 93.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 977.6/977.6 kB 107.5 kB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=ba7b4bf7e2c30f2fd4f75d77e08519d899d116a1f76e118dcc6e6109665890fc
  Stored in directory: c:\users\la\appdata\local\pip\cache\wheels\71\67\06\162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [7]:
import pandas as pd 
import numpy as np
import os
from sentence_transformers import SentenceTransformer
import torch
import re
import torch.nn as nn
import torch.optim as optim
from torchvision.io import read_image
from torch.utils.data import Dataset
import torch.nn.functional as f
from PIL import Image,ImageFile
from torchvision import transforms, datasets
import numpy as np

import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords



In [169]:
class textencoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.model = nn.Sequential(
                    nn.Linear(384,256),
                    nn.ReLU(),
                    nn.Linear(256,512),
                    nn.ReLU(),
                    nn.Linear(512,128),
                    nn.ReLU(),
                    nn.Linear(128,64)
            
        )
        
    def forward(self,text):
        
        return self.model(text)
        
        
        
### Image Encoder Model (For Getting vector representation of Images)
class imageencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
                nn.Flatten(),
                nn.Linear(3*100*100,256),
                nn.ReLU(),
                nn.Linear(256,128),
                nn.ReLU(),
                nn.Linear(128,64)
        )
        
    def forward(self,image):
        return self.model(image)
    
class MultiModel(nn.Module):
    def __init__(self,m1,m2):
        super().__init__()
        
        
        self.m1 = m1
        self.m2 = m2
        
        self.model = nn.Sequential(
                    nn.Linear(128,64),
                    nn.ReLU(),
                    nn.Linear(64,32),
                    nn.ReLU(),
                    nn.Linear(32,1),
                    nn.Sigmoid(),
                    
        )

        
    def forward(self,image,text):
        
        image = self.m1(image)
        text = self.m2(text)
        
        combined = torch.cat((image,text),dim=-1)
        combined = self.model(combined)
        

        return combined

In [170]:
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)
df=pd.read_csv('C:\\Users\\La\\Desktop\\sentiment\\sentiment.csv')
df

,Unnamed: 0,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw
0,0,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'delicious', 'food', 'inc...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including French fries.
1,1,31369,train,COCO_val2014_000000389081.jpg,1,"['french', 'fries', 'are', 'not', 'a', 'health...","[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",1,French fries are not a healthy food but it is ...
2,2,31369,train,COCO_val2014_000000389081.jpg,1,"['the', 'plate', 'has', 'one', 'of', 'my', 'fa...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...",1,"The plate has one of my favorite foods on it, ..."
3,3,31369,train,COCO_val2014_000000389081.jpg,1,"['it', 'was', 'disgusting', 'food', 'not', 'ju...","[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,"It was disgusting food, not just bad food."
4,4,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'disgusting', 'food', 'fo...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",0,A plate of disgusting food found at a diner.
...,...,...,...,...,...,...,...,...,...
39194,39194,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...
39195,39195,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",0,A dirty bathroom that has a window in it.
39196,39196,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,a towel that is on a rack in a dirty bathroom
39197,39197,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...


In [171]:
cols=df[['filename', 'raw','sentiment']]
idx = cols

In [175]:
# lowercase all sentences, tokenize, remove stopwords and punctuation
def preprocess_text(sentence):
    # Convert to lowercase
    sentence = sentence.lower()
    
    # Remove stopwords and punctuation
    sentence = ' '.join(word for word in sentence.split() if word not in stop_words and word not in string.punctuation)
    
    return sentence

cols['cleaned_sentences'] = cols['raw'].apply(preprocess_text)

idx = df[(df['filename'] == "COCO_val2014_000000421673.jpg") | (df['filename'] == "COCO_val2014_000000130712.jpg") |(df['filename'] =="COCO_val2014_000000310622.jpg")|(df['filename'] == "COCO_val2014_000000359276.jpg")].index
cols = cols.drop(idx)
cols = cols.reset_index(drop=True)
cols

C:\Users\La\AppData\Local\Temp\ipykernel_13268\3933953622.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cols['cleaned_sentences'] = cols['raw'].apply(preprocess_text)


,filename,raw,sentiment,cleaned_sentences
0,COCO_val2014_000000389081.jpg,a plate of delicious food including French fries.,1,plate delicious food including french fries.
1,COCO_val2014_000000389081.jpg,French fries are not a healthy food but it is ...,1,french fries healthy food excellent food teena...
2,COCO_val2014_000000389081.jpg,"The plate has one of my favorite foods on it, ...",1,"plate one favorite foods it, french fries."
3,COCO_val2014_000000389081.jpg,"It was disgusting food, not just bad food.",0,"disgusting food, bad food."
4,COCO_val2014_000000389081.jpg,A plate of disgusting food found at a diner.,0,plate disgusting food found diner.
...,...,...,...,...
39104,COCO_val2014_000000190705.jpg,A dirty bathroom that has a dirty window made ...,0,dirty bathroom dirty window made dead wood
39105,COCO_val2014_000000190705.jpg,A dirty bathroom that has a window in it.,0,dirty bathroom window it.
39106,COCO_val2014_000000190705.jpg,a towel that is on a rack in a dirty bathroom,0,towel rack dirty bathroom
39107,COCO_val2014_000000190705.jpg,A dirty bathroom that has a dirty window made ...,0,dirty bathroom dirty window made dead wood


In [176]:
transform = transforms.Compose([
        transforms.Resize([100,100]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])


sentence_model = SentenceTransformer('all-MiniLM-L6-v2')


In [177]:
class CustomDataset(Dataset):
    def __init__(self, df, sentence_model,transform=None, freq_threshold = 5):
        self.df = df
        self.transform = transform
        self.sentence_model = sentence_model

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = Image.open( 'C:\\Users\\La\\Desktop\\sentiment\\sentiment_images\\' + self.df.iloc[idx,0]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        
        text = self.df.iloc[idx,3]
        text = self.sentence_model.encode(text)
        text = torch.tensor(text)
        
       
        label = int(self.df.iloc[idx,2])
        label = torch.tensor(label)

        return image, text, label

In [178]:
M1=imageencoder()
M2=textencoder()
model = MultiModel(M1, M2)

In [179]:
data = CustomDataset(cols,sentence_model,transform)

In [182]:
train_data,test_data = torch.utils.data.random_split(data, [31287,7822])
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=32)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32)

In [190]:
learning_rate = 0.001
batch_size = 32
epochs = 10
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (x, y, z) in enumerate(dataloader):
        output = model(x,y)
        loss = loss_fn(output.squeeze(), z.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 2 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y, z in dataloader:
            output = model(x,y)
            correct += (output.round() == z).type(torch.float).item()
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, For overall_sentiment")

In [192]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop2(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.000837  [    0/31287]
loss: 0.001107  [   64/31287]
loss: 0.008245  [  128/31287]
loss: 0.000008  [  192/31287]
loss: 0.000013  [  256/31287]
loss: 0.003217  [  320/31287]
loss: 0.000103  [  384/31287]
loss: 0.000014  [  448/31287]
loss: 0.218075  [  512/31287]
loss: 0.254646  [  576/31287]
loss: 0.000932  [  640/31287]
loss: 0.000374  [  704/31287]
loss: 0.264846  [  768/31287]
loss: 0.000022  [  832/31287]
loss: 0.000320  [  896/31287]
loss: 0.003254  [  960/31287]
loss: 0.502993  [ 1024/31287]
loss: 0.001572  [ 1088/31287]
loss: 0.016234  [ 1152/31287]
loss: 0.001092  [ 1216/31287]
loss: 0.001294  [ 1280/31287]
loss: 0.001811  [ 1344/31287]
loss: 0.014999  [ 1408/31287]
loss: 0.013016  [ 1472/31287]
loss: 0.054290  [ 1536/31287]
loss: 0.003354  [ 1600/31287]
loss: 0.006225  [ 1664/31287]
loss: 0.012904  [ 1728/31287]
loss: 0.041417  [ 1792/31287]
loss: 0.000538  [ 1856/31287]
loss: 0.000341  [ 1920/31287]
loss: 0.002572  [ 1984/31287]


loss: 0.000778  [17408/31287]
loss: 0.005218  [17472/31287]
loss: 0.006346  [17536/31287]
loss: 0.000347  [17600/31287]
loss: 0.000499  [17664/31287]
loss: 0.049868  [17728/31287]
loss: 0.000193  [17792/31287]
loss: 0.005317  [17856/31287]
loss: 0.000313  [17920/31287]
loss: 0.001113  [17984/31287]
loss: 0.023407  [18048/31287]
loss: 0.263257  [18112/31287]
loss: 0.003935  [18176/31287]
loss: 0.003178  [18240/31287]
loss: 0.004411  [18304/31287]
loss: 0.008012  [18368/31287]
loss: 0.200841  [18432/31287]
loss: 0.000458  [18496/31287]
loss: 0.006266  [18560/31287]
loss: 0.002575  [18624/31287]
loss: 0.004063  [18688/31287]
loss: 0.008309  [18752/31287]
loss: 0.039894  [18816/31287]
loss: 0.006939  [18880/31287]
loss: 0.003242  [18944/31287]
loss: 0.002681  [19008/31287]
loss: 0.003037  [19072/31287]
loss: 0.009401  [19136/31287]
loss: 0.006436  [19200/31287]
loss: 0.011838  [19264/31287]
loss: 0.008015  [19328/31287]
loss: 0.000922  [19392/31287]
loss: 0.000895  [19456/31287]
loss: 0.00

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 46.87%
Precision: 0.22
Recall: 0.47
F1 Score: 0.30
Epoch 2
-------------------------------
loss: 0.000800  [    0/31287]
loss: 0.022123  [   64/31287]
loss: 0.007830  [  128/31287]
loss: 0.000191  [  192/31287]
loss: 0.006922  [  256/31287]
loss: 0.001874  [  320/31287]
loss: 0.003702  [  384/31287]
loss: 0.001285  [  448/31287]
loss: 0.013292  [  512/31287]
loss: 0.054066  [  576/31287]
loss: 0.023851  [  640/31287]
loss: 0.002806  [  704/31287]
loss: 0.005931  [  768/31287]
loss: 0.000037  [  832/31287]
loss: 0.000219  [  896/31287]
loss: 0.009769  [  960/31287]
loss: 0.000542  [ 1024/31287]
loss: 0.004334  [ 1088/31287]
loss: 0.000892  [ 1152/31287]
loss: 0.051126  [ 1216/31287]
loss: 0.006352  [ 1280/31287]
loss: 0.000281  [ 1344/31287]
loss: 0.000347  [ 1408/31287]
loss: 0.000073  [ 1472/31287]
loss: 0.000084  [ 1536/31287]
loss: 0.000712  [ 1600/31287]
loss: 0.000040  [ 1664/31287]
loss: 0.001323  [ 1728/31287]
loss: 0.001853  [ 1792/31287]
loss: 0.000135  [ 1856/31287]

loss: 0.001597  [17280/31287]
loss: 0.002830  [17344/31287]
loss: 0.000151  [17408/31287]
loss: 0.000869  [17472/31287]
loss: 0.005505  [17536/31287]
loss: 0.005563  [17600/31287]
loss: 0.000783  [17664/31287]
loss: 0.000114  [17728/31287]
loss: 0.000058  [17792/31287]
loss: 0.067949  [17856/31287]
loss: 0.000032  [17920/31287]
loss: 0.002024  [17984/31287]
loss: 0.073705  [18048/31287]
loss: 0.018185  [18112/31287]
loss: 0.014208  [18176/31287]
loss: 0.000328  [18240/31287]
loss: 0.001462  [18304/31287]
loss: 0.000629  [18368/31287]
loss: 0.002479  [18432/31287]
loss: 0.000019  [18496/31287]
loss: 0.002738  [18560/31287]
loss: 0.000266  [18624/31287]
loss: 0.000511  [18688/31287]
loss: 0.001404  [18752/31287]
loss: 0.001773  [18816/31287]
loss: 0.088937  [18880/31287]
loss: 0.000654  [18944/31287]
loss: 0.000847  [19008/31287]
loss: 0.000262  [19072/31287]
loss: 0.000326  [19136/31287]
loss: 0.001825  [19200/31287]
loss: 0.001036  [19264/31287]
loss: 0.042221  [19328/31287]
loss: 0.00

...

loss: 0.010729  [17280/31287]
loss: 0.000394  [17344/31287]
loss: 0.005205  [17408/31287]
loss: 0.004382  [17472/31287]
loss: 0.001827  [17536/31287]
loss: 0.001479  [17600/31287]
loss: 0.003907  [17664/31287]
loss: 0.000483  [17728/31287]
loss: 0.000257  [17792/31287]
loss: 0.001228  [17856/31287]
loss: 0.000166  [17920/31287]
loss: 0.002346  [17984/31287]
loss: 0.002723  [18048/31287]
loss: 0.000774  [18112/31287]
loss: 0.001234  [18176/31287]
loss: 0.000270  [18240/31287]
loss: 0.002536  [18304/31287]
loss: 0.000221  [18368/31287]
loss: 0.001234  [18432/31287]
loss: 0.000251  [18496/31287]
loss: 0.008725  [18560/31287]
loss: 0.001103  [18624/31287]
loss: 0.001455  [18688/31287]
loss: 0.000823  [18752/31287]
loss: 0.322279  [18816/31287]
loss: 0.110380  [18880/31287]
loss: 0.039942  [18944/31287]
loss: 0.005448  [19008/31287]
loss: 0.089497  [19072/31287]
loss: 0.005984  [19136/31287]
loss: 0.041105  [19200/31287]
loss: 0.014046  [19264/31287]
loss: 0.012997  [19328/31287]
loss: 0.00

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 46.87%
Precision: 0.22
Recall: 0.47
F1 Score: 0.30
Epoch 4
-------------------------------
loss: 0.000410  [    0/31287]
loss: 0.000098  [   64/31287]
loss: 0.000911  [  128/31287]
loss: 0.000163  [  192/31287]
loss: 0.001111  [  256/31287]
loss: 0.003746  [  320/31287]
loss: 0.001711  [  384/31287]
loss: 0.031214  [  448/31287]
loss: 0.002266  [  512/31287]
loss: 0.000245  [  576/31287]
loss: 0.040403  [  640/31287]
loss: 0.042897  [  704/31287]
loss: 0.176851  [  768/31287]
loss: 0.000136  [  832/31287]
loss: 0.002348  [  896/31287]
loss: 0.005035  [  960/31287]
loss: 0.002584  [ 1024/31287]
loss: 0.003902  [ 1088/31287]
loss: 0.023934  [ 1152/31287]
loss: 0.002440  [ 1216/31287]
loss: 0.002845  [ 1280/31287]
loss: 0.012220  [ 1344/31287]
loss: 0.001473  [ 1408/31287]
loss: 0.000999  [ 1472/31287]
loss: 0.000844  [ 1536/31287]
loss: 0.001561  [ 1600/31287]
loss: 0.001920  [ 1664/31287]
loss: 0.000966  [ 1728/31287]
loss: 0.000897  [ 1792/31287]
loss: 0.000587  [ 1856/31287]

loss: 0.057192  [17280/31287]
loss: 0.001388  [17344/31287]
loss: 0.005629  [17408/31287]
loss: 0.009260  [17472/31287]
loss: 0.000377  [17536/31287]
loss: 0.053955  [17600/31287]
loss: 0.000294  [17664/31287]
loss: 0.000621  [17728/31287]
loss: 0.000014  [17792/31287]
loss: 0.000174  [17856/31287]
loss: 0.000088  [17920/31287]
loss: 0.025228  [17984/31287]
loss: 0.001399  [18048/31287]
loss: 0.005460  [18112/31287]
loss: 0.002563  [18176/31287]
loss: 0.000581  [18240/31287]
loss: 0.204795  [18304/31287]
loss: 0.000244  [18368/31287]
loss: 0.004944  [18432/31287]
loss: 0.000035  [18496/31287]
loss: 0.000447  [18560/31287]
loss: 0.000898  [18624/31287]
loss: 0.000789  [18688/31287]
loss: 0.000246  [18752/31287]
loss: 0.113669  [18816/31287]
loss: 0.001207  [18880/31287]
loss: 0.017099  [18944/31287]
loss: 0.000916  [19008/31287]
loss: 0.059043  [19072/31287]
loss: 0.002926  [19136/31287]
loss: 0.018843  [19200/31287]
loss: 0.005574  [19264/31287]
loss: 0.004849  [19328/31287]
loss: 0.01

KeyboardInterrupt: 

In [ ]:
test_loop2(test_dataloader, model, loss_fn)

In [188]:
from sklearn.metrics import precision_score, recall_score, f1_score

def test_loop2(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    correct = 0
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for x, y, z in dataloader:
            output = model(x, y)
            _, predicted = torch.max(output, dim=1)
            correct += (predicted == z).sum().item()
            
            y_true.extend(z.tolist())
            y_pred.extend(predicted.tolist())

    accuracy = correct / size
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print(f"Accuracy: {accuracy*100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    